In [1]:
# Copyright 2024 Ethan Lee Christensen
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import re
import os
import docx
import openai
import requests

from docx.shared import Inches
from docx2pdf import convert
from bruhcolor import bruhcolored as bc
from dotenv import load_dotenv

load_dotenv()

client = openai.OpenAI()
# client = openai.AzureOpenAI(
#     api_key=os.getenv("OPENAI_API_KEY"),
#     api_version=os.getenv("OPENAI_API_VERSION"),
#     azure_endpoint=os.getenv("OPENAI_API_BASE")
# )

c:\Users\ethan\AppData\Local\pypoetry\Cache\virtualenvs\gptbookmaker-9bZx-Cay-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def complete(system_prompt: str, user_prompt: str, model: str = os.getenv("MODEL")) -> str:
    try:
        return client.chat.completions.create(
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.5,
            max_tokens=1500,
            model=model
        ).choices[0].message.content
    except Exception as e:
        print(f"ERROR: {e}")
        return None

In [3]:
def create_cover_art(story_type: str, target_audience: str, image_save_path: str) -> None:
    system_prompt = """
You are an expert image prompt creator. You create prompts for people that will be used to create images with tools like DALL-E-2 or MidJourney.
These prompts are used to create cover art for books. You will be provided what the book is about and the target audience.
- You will be given what the story is about surrounded in four hashtags (####) (e.g. #### [Book info here] ####)
- You will be given the target audience of the book surrounded in four percent symbols (%%%%) (e.g. %%%% 5 - 7 year olds %%%%)

Return back a prompt that can be used with DALL-E-2 or Midjourney that would create a cover art for this book. Make sure it is detailed!
"""

    user_prompt = """
Book Info: #### ${book_info} ####
Target Audience: %%%% ${target_audience} %%%%
[Prompt for iamge creation tool here]
"""
    
    prompt = complete(
        system_prompt=system_prompt,
        user_prompt=user_prompt.replace(
            "${book_info}", story_type).replace(
                "${target_audience}", target_audience),
        model="gpt-4"
    )
    
    print(bc(f"Creating cover art for book at '{image_save_path + '/cover_art.png'}' with GPT-4 generated prompt of:\n{prompt}", color=207))
    
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        quality="standard",
        n=1
    )
    
    image_url = response.data[0].url
    image_data = requests.get(image_url).content
    with open(image_save_path + "/cover_art.png", "wb") as image_file:
        image_file.write(image_data)

In [4]:
def save_to_pdf(base_path: str, story_type: str) -> None:
    
    main_title = complete(
        "You are an expert book title maker. Give some info about the book, create a simple 5 word or less title.",
        story_type
    )
    
    main_title = re.sub(r"[^A-Za-z0-9 ]", "", main_title).replace(" ", "_")
    
    print(bc(f"Creating '{base_path}/{main_title}.pdf'", color=72))
    doc = docx.Document()
    
    title = os.path.basename(base_path)
    doc.add_paragraph(title.replace("_", " "), "Title")
    doc.add_picture(f"{base_path}/cover_art.png", width=Inches(6))
        
    doc.add_page_break()
    
    for chapter_folder in os.listdir(base_path):
        chapter_path = os.path.join(base_path, chapter_folder)
        if os.path.isdir(chapter_path):
            doc.add_paragraph(chapter_folder.replace("_", " "), "Title")
            for page_file in os.listdir(chapter_path):
                page_path = os.path.join(chapter_path, page_file)
                if os.path.isfile(page_path) and page_file.endswith(".txt"):
                    with open(page_path, "r") as file:
                        doc.add_paragraph("".join(file.readlines()[2:]))
        doc.add_page_break()
                        
    doc.save(f"{base_path}/{main_title}.docx")
    try:
        convert(f"{base_path}/{main_title}.docx", f"{base_path}/{main_title}.pdf")
        os.remove(f"{base_path}/{main_title}.docx")
    except Exception as e:
        print(e)

In [5]:
def generate_outline(story_type: str, target_audience: str):
    system_prompt = """
You are an expert story outliner.
You will be provided a type of story and target audience.
Your task is to take these two items and create a story outline that has 5 main plot points.

You will be given the story type surrounded in four hashtags (####). As an example, #### <story type> ####.    
You will be given the target audience surrounded in four dollar signs ($$$$). As an example, $$$$ <story type> $$$$.

Rules:
- You MUST generate a rich story outline with 3 to 5 main plot points.
- You MUST lead main points with a single dash.
- You MUST provide 5 to 10 sub points for each main point that explains the story line for the main point.
- You MUST lead sub points with two dashes.


Example Input:
Story Type:
####
<story type here>
####

Target Audience:
$$$$
<target audience here>
$$$$


Example Output:
- Point one
-- sub point 1-1
-- sub point 1-2
-- sub point 1-3
. . . 
"""

    user_prompt = """
Story Type:
####
${story_type}
####

Target Audience:
$$$$
${target_audience}
$$$$
"""

    response = complete(
        system_prompt=system_prompt,
        user_prompt=user_prompt.replace("${story_type}", story_type).replace("${target_audience}", target_audience)
    ).split("\n")

    response = [val for val in response if val.strip() != ""]

    chapter_points = {}

    current_chapter = ""

    for val in response:
        if re.match(r"\-\s.*", val):
            current_chapter = val.split("- ")[1]
            chapter_points[current_chapter] = []
        elif re.match(r"\-\-\s.*", val):
            chapter_points[current_chapter].append(val.split("-- ")[1])
        else:
            print("ERROR")

    return chapter_points

In [6]:
def chapter_page_generator(
    story_type: str,
    target_audience: str,
    chapter_title: str,
    chapter_point: str,
    previous_knowledge: str,
    completed_chapters: str,
    last_paragraph: str, 
    chapter_save_path: str,
    page_number: int,
    last_chapter: bool = False,
    last_page: bool = False
) -> str:
    system_prompt = """
You are an expert partial story generator. Your task is to creatively expand upon provided story details, crafting a coherent and engaging single, multi-paragraph page to add to the story.

Please follow these guidelines:

- Story Title/Purpose: Identified by four hashtags (e.g., #### Story Title ####).
- Target Audience: Specified within four @ symbols (e.g., @@@@ Adults @@@@).
- Chapter Title: Highlighted by four dollar signs (e.g., $$$$ Chapter Title $$$$).
- Page Overview: Enclosed in four asterisks (e.g., **** Page Details ****).
- Optional - Previous Page Knowledge: Provided within four exclamation points (e.g., !!!! Previous Page Summary !!!!).
- Last Chapter Indicator: A boolean flag (True/False) to indicate if it's the last chapter.
- Last Page Indicator: A boolean flag (True/False) to indicate if it's the last page of the story.

Your Task:

- Using the provided details, create the next page of the story.
- Ensure continuity with previous content if available.
- Aim for a length of approximately [specify word/paragraph count].
- Be creative while adhering to the guidelines.
- If 'Last Chapter' is True, ensure the chapter concludes appropriately.
- If 'Last Page' is True, bring the story to a satisfying conclusion.

For example:
#### The Lost City ####
@@@@ Young Adults @@@@
$$$$ The Secret Door $$$$
**** Write about the protagonist discovering a hidden door in the ruins ****
!!!! 
In the previous page, the protagonist was exploring ancient ruins. 
!!!!
Last Chapter: True
Last Page of Chapter: False
[Your story content here]

Note: Do not include the chapter title in your story content.
"""

    user_prompt = """
#### ${story_title} ####
@@@@ ${target_audience} @@@@
$$$$ ${chapter_title} $$$$
**** ${page_to_create} ****
!!!!
${last_paragraph}
!!!!
Last Chapter: ${last_chapter}
Last Page of Chapter: ${last_page}
[Your story content here]
"""

    if previous_knowledge == "":
        previous_knowledge = "No Previous Knowledge."
    if completed_chapters == "":
        completed_chapters = "No Previously Completed Chapters."
    if last_paragraph == "":
        last_paragraph = "No Last Paragraph - Start of a new chatper."

    filled_user_prompt = (
        user_prompt.replace("${story_title}", story_type)
        .replace("${target_audience}", target_audience)
        .replace("${chapter_title}", chapter_title)
        .replace("${page_to_create}", chapter_point)
        .replace("${previous_knowledge}", previous_knowledge)
        .replace("${completed_chapters}", completed_chapters)
        .replace("${last_paragraph}", last_paragraph)
        .replace("${last_chapter}", "is" if last_chapter else "is not")
        .replace("${last_page}", "is" if last_page else "is not")
    )

    new_page = complete(system_prompt=system_prompt, user_prompt=filled_user_prompt)
    
    last_paragraph = [val for val in new_page.split("\n") if val.strip() != ""][-1]

    page_path = f"{chapter_save_path}/page_{page_number}.txt"

    with open(page_path, "w") as file:
        print(bc(f"Creating Chapter Page: '{page_path}'", color=87))
        file.write(
            f"Story Type: {story_type}\n"
            + f"Chapter Title: {chapter_title}\n\n"
            + new_page
            + "\n"
        )

    return (
        previous_knowledge
        + f"Completed Page: {chapter_title} - Page {page_number} - {chapter_point}\n", last_paragraph
    )

In [7]:
def story_generator(story_type: str, target_audience: str, story_outline: dict[str, list[str]]) -> None:
    
    cleaned_story_type = re.sub(r"[^A-Za-z0-9 ]", "", story_type).replace(" ", "_")
    
    if not os.path.exists("./stories"):
        print(bc("Creating 'stories' base folder", color=82))
        os.mkdir("./stories")
    
    if not os.path.exists(f"./stories/{cleaned_story_type}"):
        print(bc(f"Creating 'stories/{cleaned_story_type}' story folder", color=72))
        os.mkdir(f"./stories/{cleaned_story_type}")
    
    base_path = f"./stories/{cleaned_story_type}"
    
    create_cover_art(
        story_type=story_type,
        target_audience=target_audience,
        image_save_path=base_path
    )
    
    chapter_number = 0
    
    completed_chapters = ""
    
    for chapter_num, chapter_val in enumerate(story_outline.items()):
        last_chapter = chapter_num == len(story_outline) - 1
        chapter, chapter_points = chapter_val
        chapter_number += 1
        previous_chapter_knowledge = ""
        last_paragraph = ""
        cleaned_chapter_name = f"{chapter_number}_" + re.sub(r"[^A-Za-z0-9 ]", "", chapter).replace(" ", "_")
        chapter_page_save_path = f"./stories/{cleaned_story_type}/{cleaned_chapter_name}"
        print(bc(f"Creating Chapter Folder '{chapter_page_save_path}'", color=206))
        os.mkdir(chapter_page_save_path)
        for idx, point in enumerate(chapter_points):
            last_page = idx == len(chapter_points) - 1
            previous_chapter_knowledge, last_paragraph = chapter_page_generator(
                story_type=story_type,
                target_audience=target_audience,
                chapter_title=chapter,
                chapter_point=point,
                previous_knowledge=previous_chapter_knowledge,
                completed_chapters=completed_chapters,
                last_paragraph=last_paragraph,
                chapter_save_path=chapter_page_save_path,
                page_number=idx,
                last_chapter=last_chapter,
                last_page=last_page
            )
        completed_chapters += f"Previously Completed Chapter Title - {chapter}\n"

    save_to_pdf(
        base_path=base_path,
        story_type=cleaned_story_type
    )

In [8]:
story_type = "A short story about the life of James B. Cornell, a WWII soldier."
target_audience = "college students"

story_outline = generate_outline(
    story_type=story_type,
    target_audience=target_audience
)

story_generator(
    story_type=story_type,
    target_audience=target_audience,
    story_outline=story_outline
)

Creating 'stories/A_short_story_about_the_life_of_James_B_Cornell_a_WWII_soldier' story folder
Creating cover art for book at './stories/A_short_story_about_the_life_of_James_B_Cornell_a_WWII_soldier/cover_art.png' with GPT-4 generated prompt of:
"A vintage style book cover featuring a determined and brave young soldier named James B. Cornell in WWII uniform, standing against a backdrop of a war-torn battlefield. He is holding a helmet under his arm, looking into the distance with hope and resilience. The atmosphere should be dramatic yet inspiring, with muted, sepia-toned colors to evoke a sense of history and nostalgia. The title of the book 'Life of a WWII Soldier: James B. Cornell' should be prominently displayed in a bold, classic font."
Creating Chapter Folder './stories/A_short_story_about_the_life_of_James_B_Cornell_a_WWII_soldier/1_James_B_Cornell_enlists_in_the_army_during_WWII'
Creating Chapter Page: './stories/A_short_story_about_the_life_of_James_B_Cornell_a_WWII_soldier/1